## Imports

In [365]:
import pandas as pd
import numpy as np
import hvplot.pandas
import yfinance as yf
import datetime


## Data Collection

In [366]:
# Define the stock symbol and create a Yahoo Finance ticker object
symbol = "CORN"
ticker = yf.Ticker(symbol)

# Get historical stock data for the last 5 years
historical_data = ticker.history(period="5y")

# Print the first few rows of the historical data
print(historical_data.head())

                            Open   High    Low  Close  Volume  Dividends  \
Date                                                                       
2018-09-13 00:00:00-04:00  15.76  15.80  15.63  15.70   55800        0.0   
2018-09-14 00:00:00-04:00  15.65  15.73  15.65  15.71   31600        0.0   
2018-09-17 00:00:00-04:00  15.68  15.68  15.58  15.59   55600        0.0   
2018-09-18 00:00:00-04:00  15.56  15.56  15.39  15.40   98700        0.0   
2018-09-19 00:00:00-04:00  15.41  15.54  15.41  15.50   49900        0.0   

                           Stock Splits  Capital Gains  
Date                                                    
2018-09-13 00:00:00-04:00           0.0            0.0  
2018-09-14 00:00:00-04:00           0.0            0.0  
2018-09-17 00:00:00-04:00           0.0            0.0  
2018-09-18 00:00:00-04:00           0.0            0.0  
2018-09-19 00:00:00-04:00           0.0            0.0  


## Daily Returns

In [367]:
# Calculate daily returns
historical_data['Daily_Return'] = historical_data['Close'].pct_change()

# Print the first few rows of the data with daily returns
print(historical_data.head())

                            Open   High    Low  Close  Volume  Dividends  \
Date                                                                       
2018-09-13 00:00:00-04:00  15.76  15.80  15.63  15.70   55800        0.0   
2018-09-14 00:00:00-04:00  15.65  15.73  15.65  15.71   31600        0.0   
2018-09-17 00:00:00-04:00  15.68  15.68  15.58  15.59   55600        0.0   
2018-09-18 00:00:00-04:00  15.56  15.56  15.39  15.40   98700        0.0   
2018-09-19 00:00:00-04:00  15.41  15.54  15.41  15.50   49900        0.0   

                           Stock Splits  Capital Gains  Daily_Return  
Date                                                                  
2018-09-13 00:00:00-04:00           0.0            0.0           NaN  
2018-09-14 00:00:00-04:00           0.0            0.0      0.000637  
2018-09-17 00:00:00-04:00           0.0            0.0     -0.007638  
2018-09-18 00:00:00-04:00           0.0            0.0     -0.012187  
2018-09-19 00:00:00-04:00           0.0  

In [368]:
# Convert to DataFrame and Reset Index
df = pd.DataFrame(historical_data)
df = df.reset_index()
df = df.drop(columns = ["Open", "High", "Low", "Volume", "Dividends", "Stock Splits", "Capital Gains"])
df.head()

,Date,Close,Daily_Return
0,2018-09-13 00:00:00-04:00,15.70,NaN
1,2018-09-14 00:00:00-04:00,15.71,0.000637
2,2018-09-17 00:00:00-04:00,15.59,-0.007638
3,2018-09-18 00:00:00-04:00,15.40,-0.012187
4,2018-09-19 00:00:00-04:00,15.50,0.006494


## Annualized Returns

In [369]:

# Calculate the total return over the period
total_return = (1 + df['Daily_Return']).prod() - 1

# Determine the number of years in the investment horizon
years = (df['Date'].max() - df['Date'].min()).days / 365

# Calculate annualized return using the formula: ((1 + Total Return)^(1 / Years) - 1)
annualized_return = (pow(1 + total_return, 1 / years) - 1) * 100 

print(years)
print(annualized_return)

5.0
6.970768417538853


## Cumulative Returns

In [370]:
# Calculate Cumulative Return and add as a Column
df['Cumulative_Return'] = (1 + df['Daily_Return']).cumprod() - 1

df.head()

,Date,Close,Daily_Return,Cumulative_Return
0,2018-09-13 00:00:00-04:00,15.70,NaN,NaN
1,2018-09-14 00:00:00-04:00,15.71,0.000637,0.000637
2,2018-09-17 00:00:00-04:00,15.59,-0.007638,-0.007006
3,2018-09-18 00:00:00-04:00,15.40,-0.012187,-0.019108
4,2018-09-19 00:00:00-04:00,15.50,0.006494,-0.012739


## Annual Volatility 

In [371]:
# Calculate the standard deviation of daily returns
daily_volatility = df['Daily_Return'].std()

# Annualize the volatility
annual_volatility = daily_volatility * np.sqrt(252)

print(daily_volatility)
print(annual_volatility)

0.014008746436512768
0.2223819555046504


## Sharpe Ratio

In [372]:
# Calculate the average daily return
average_daily_return = df['Daily_Return'].mean()

# Assume a daily risk free rate
daily_risk_free_rate = 0.001

# Calculate the Sharpe ratio
sharpe_ratio = (average_daily_return - daily_risk_free_rate) / daily_volatility

print(average_daily_return)
print(sharpe_ratio)

0.00036626871668886407
-0.045238257840070684


## Sortino Ratio

In [373]:
# Calculate downside risk (standard deviation of negative returns)
negative_returns = df[df['Daily_Return'] < 0]['Daily_Return']
downside_risk = negative_returns.std()

# Calculate the Sortino ratio
sortino_ratio = (average_daily_return - daily_risk_free_rate) / downside_risk

print(sortino_ratio)

-0.06644070617309594


# Trading Algorithm

In [374]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

## Create Short and Long Window 

In [375]:
df = df.set_index(["Date"])

df['SMA_Short'] = df['Close'].rolling(window=45).mean()
df['SMA_Long'] = df['Close'].rolling(window=90).mean()
df.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long
Date,,,,,
2018-09-13 00:00:00-04:00,15.70,NaN,NaN,NaN,NaN
2018-09-14 00:00:00-04:00,15.71,0.000637,0.000637,NaN,NaN
2018-09-17 00:00:00-04:00,15.59,-0.007638,-0.007006,NaN,NaN
2018-09-18 00:00:00-04:00,15.40,-0.012187,-0.019108,NaN,NaN
2018-09-19 00:00:00-04:00,15.50,0.006494,-0.012739,NaN,NaN


In [376]:
# Create a binary target variable: 1 for buy signal, -1 for sell signal

df['Signal'] = np.where(df['SMA_Short'] > df['SMA_Long'], 1, -1)
df.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long,Signal
Date,,,,,,
2018-09-13 00:00:00-04:00,15.70,NaN,NaN,NaN,NaN,-1
2018-09-14 00:00:00-04:00,15.71,0.000637,0.000637,NaN,NaN,-1
2018-09-17 00:00:00-04:00,15.59,-0.007638,-0.007006,NaN,NaN,-1
2018-09-18 00:00:00-04:00,15.40,-0.012187,-0.019108,NaN,NaN,-1
2018-09-19 00:00:00-04:00,15.50,0.006494,-0.012739,NaN,NaN,-1


In [377]:
# Drop NaN values and shift target variable by one day
df['Signal'] = df['Signal'].shift(-1)
df = df.dropna()
df.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long,Signal
Date,,,,,,
2019-01-23 00:00:00-05:00,16.280001,0.001846,0.036943,16.212000,16.193667,1.0
2019-01-24 00:00:00-05:00,16.240000,-0.002457,0.034395,16.213556,16.199667,1.0
2019-01-25 00:00:00-05:00,16.309999,0.004310,0.038853,16.217778,16.206333,1.0
2019-01-28 00:00:00-05:00,16.299999,-0.000613,0.038217,16.224000,16.214222,1.0
2019-01-29 00:00:00-05:00,16.260000,-0.002454,0.035669,16.230222,16.223778,1.0


In [378]:
# Prepare data for machine learning
X = df.drop(columns = (["Close", "Daily_Return", "Cumulative_Return", "Signal"]))
y = df["Signal"]
display(X)
display(y)


,SMA_Short,SMA_Long
Date,,
2019-01-23 00:00:00-05:00,16.212000,16.193667
2019-01-24 00:00:00-05:00,16.213556,16.199667
2019-01-25 00:00:00-05:00,16.217778,16.206333
2019-01-28 00:00:00-05:00,16.224000,16.214222
2019-01-29 00:00:00-05:00,16.230222,16.223778
...,...,...
2023-09-05 00:00:00-04:00,22.820889,23.261778
2023-09-06 00:00:00-04:00,22.818667,23.249000
2023-09-07 00:00:00-04:00,22.822000,23.237111


Date
2019-01-23 00:00:00-05:00    1.0
2019-01-24 00:00:00-05:00    1.0
2019-01-25 00:00:00-05:00    1.0
2019-01-28 00:00:00-05:00    1.0
2019-01-29 00:00:00-05:00    1.0
                            ... 
2023-09-05 00:00:00-04:00   -1.0
2023-09-06 00:00:00-04:00   -1.0
2023-09-07 00:00:00-04:00   -1.0
2023-09-08 00:00:00-04:00   -1.0
2023-09-11 00:00:00-04:00   -1.0
Name: Signal, Length: 1167, dtype: float64

## Run Logistic Regression model on Trading Data

In [379]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [380]:
# Scale the data and transform
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [381]:
# Build and train a machine learning model
model = LogisticRegression(random_state = 1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [382]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
testing_report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(testing_report)

Accuracy: 0.9358974358974359
              precision    recall  f1-score   support

        -1.0       0.88      1.00      0.94       112
         1.0       1.00      0.88      0.93       122

    accuracy                           0.94       234
   macro avg       0.94      0.94      0.94       234
weighted avg       0.94      0.94      0.94       234



## Backtest the Logistic Regression Model

In [385]:
# Create a new empty predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)
predictions_df["Predicted Signal"] = y_pred
predictions_df["Actual Returns"] = df["Daily_Return"]
predictions_df["Trading Algorithm Returns"] = predictions_df["Actual Returns"] * predictions_df["Predicted Signal"]
predictions_df

,Predicted Signal,Actual Returns,Trading Algorithm Returns
Date,,,
2020-11-12 00:00:00-05:00,1.0,-0.016277,-0.016277
2020-04-15 00:00:00-04:00,-1.0,-0.013688,0.013688
2019-11-29 00:00:00-05:00,-1.0,0.013259,-0.013259
2022-11-23 00:00:00-05:00,1.0,0.005331,0.005331
2021-02-23 00:00:00-05:00,1.0,0.005217,0.005217
...,...,...,...
2021-03-08 00:00:00-05:00,1.0,-0.002882,-0.002882
2022-10-20 00:00:00-04:00,1.0,0.008856,0.008856
2019-05-30 00:00:00-04:00,-1.0,0.026538,-0.026538


In [387]:
#Calcualte the cumulative returns for the actual returns and trading algorithm returns
cumulative_df = pd.DataFrame(index=X_test.index)
cumulative_df['Actual Cumulative Returns'] = (1 + predictions_df[['Actual Returns']]).cumprod()
cumulative_df['Algo Cumulative Returns'] = (1 + predictions_df[['Trading Algorithm Returns']]).cumprod()

#Review the cumulative returns dataframe
cumulative_df

,Actual Cumulative Returns,Algo Cumulative Returns
Date,,
2020-11-12 00:00:00-05:00,0.983723,0.983723
2020-04-15 00:00:00-04:00,0.970258,0.997187
2019-11-29 00:00:00-05:00,0.983122,0.983966
2022-11-23 00:00:00-05:00,0.988364,0.989212
2021-02-23 00:00:00-05:00,0.993520,0.994373
...,...,...
2021-03-08 00:00:00-05:00,1.075227,0.930594
2022-10-20 00:00:00-04:00,1.084749,0.938836
2019-05-30 00:00:00-04:00,1.113536,0.913921


## Backtest the Actual Trading Strategy

In [390]:
#Create the dataframe for the Actual Cumualtive Returns
algo_backtest = pd.concat([df, cumulative_df], axis='columns', join='inner')

#Drop the Algo Cumulative Returns Column
algo_backtest.drop(columns='Algo Cumulative Returns', inplace=True)

# Calculate the points in time at which a position should be taken, 1 or -1
algo_backtest['Entry/Exit'] = algo_backtest['Signal'].diff()
pd.set_option('display.max_rows', None)

#Review the dataframe
algo_backtest.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long,Signal,Actual Cumulative Returns,Entry/Exit
Date,,,,,,,,
2019-01-31 00:00:00-05:00,16.190001,-0.009786,0.031210,16.243333,16.238111,1.0,0.887652,NaN
2019-02-04 00:00:00-05:00,16.280001,0.002463,0.036943,16.262889,16.244333,1.0,0.912564,0.0
2019-02-06 00:00:00-05:00,16.309999,-0.001225,0.038853,16.276222,16.249222,1.0,1.061356,0.0
2019-02-20 00:00:00-05:00,16.010000,0.004392,0.019745,16.224000,16.242444,-1.0,0.906124,-2.0
2019-03-04 00:00:00-05:00,15.870000,0.006980,0.010828,16.163556,16.199667,-1.0,0.864273,0.0


In [395]:
# Set the intial capital 
initial_capital = float(100000)

# Set the share size
share_size = 500

#Take the share position where the dual moving average crossover is 1
algo_backtest['Position'] = share_size * algo_backtest['Signal']

# Find the points in time where a 500 share position is bought or sold
algo_backtest["Entry/Exit Position"] = algo_backtest["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
algo_backtest["Portfolio Holdings"] = (
    algo_backtest["Close"] * algo_backtest["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
algo_backtest["Portfolio Cash"] = (
    initial_capital - (algo_backtest["Close"] * algo_backtest["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
algo_backtest["Portfolio Total"] = (
   algo_backtest["Portfolio Cash"] + algo_backtest["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
algo_backtest["Portfolio Daily Returns"] = algo_backtest["Portfolio Total"].pct_change()

# Calculate the cumulative returns
algo_backtest["Portfolio Cumulative Returns"] = (
    1 + algo_backtest["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
algo_backtest.head()

,Close,Daily_Return,Cumulative_Return,SMA_Short,SMA_Long,Signal,Actual Cumulative Returns,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,
2019-01-31 00:00:00-05:00,16.190001,-0.009786,0.031210,16.243333,16.238111,1.0,0.887652,NaN,500.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-04 00:00:00-05:00,16.280001,0.002463,0.036943,16.262889,16.244333,1.0,0.912564,0.0,500.0,0.0,0.000000,100000.000000,100000.000000,NaN,NaN
2019-02-06 00:00:00-05:00,16.309999,-0.001225,0.038853,16.276222,16.249222,1.0,1.061356,0.0,500.0,0.0,0.000000,100000.000000,100000.000000,0.0000,0.0000
2019-02-20 00:00:00-05:00,16.010000,0.004392,0.019745,16.224000,16.242444,-1.0,0.906124,-2.0,-500.0,-1000.0,-16010.000229,116010.000229,100000.000000,0.0000,0.0000
2019-03-04 00:00:00-05:00,15.870000,0.006980,0.010828,16.163556,16.199667,-1.0,0.864273,0.0,-500.0,0.0,-15869.999886,116010.000229,100140.000343,0.0014,0.0014


## Actual Portfolio Evaluation Metrix

In [401]:
# Create the list of the metric names
metrics1 = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns1 = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
algo_evaluation_df = pd.DataFrame(index=metrics1, columns=columns1)

algo_evaluation_df.loc['Annualized Return'] = (
    algo_backtest['Portfolio Daily Returns'].mean() * 252
)

# Calculate the Cumulative returns metric
algo_evaluation_df.loc['Cumulative Returns'] = algo_backtest['Portfolio Cumulative Returns'][-1]

# Calculate the Annual volatility metric
algo_evaluation_df.loc['Annual Volatility'] = (
    algo_backtest['Portfolio Daily Returns'].std() * np.sqrt(252)
)
# Calculate the Sharpe ratio
algo_evaluation_df.loc['Sharpe Ratio'] = (
    algo_backtest['Portfolio Daily Returns'].mean() * 252) / (
    algo_backtest['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = algo_backtest[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0, 
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate the annualized return value
annualized_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)

# Divide the annualized return value by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
algo_evaluation_df.loc['Sortino Ratio'] = sortino_ratio

# Review the portfolio evaluation DataFrame
algo_evaluation_df

,Backtest
Annualized Return,0.01586
Cumulative Returns,0.0121
Annual Volatility,0.074985
Sharpe Ratio,0.211506
Sortino Ratio,0.309631


## Export to CSV

In [399]:
# Export the DataFrame to a CSV file
cumulative_df.to_csv('Farmland_ETF_Returns.csv', index=False)
algo_evaluation_df.to_csv("Farmland_Backtest.csv", index=False)